In [ ]:
%pylab inline

In [ ]:
import sys
import os
import mmap

#shmem_boxes=mmap.mmap(-1,MEM_LEN_BOXES,"NW_BUFFER_BOXES")

sys.path.append("..")
import extract_memory
layout=extract_memory.get_header_format('../memory_layout.h')

In [ ]:
from multiprocessing import shared_memory

NAME='NW_SRC0_HDR'
shmem_main = shared_memory.SharedMemory(name=NAME)

layout_log=extract_memory.get_header_format('../layout_log.h')
NAME='NW_LOG'
#shm_log = shared_memory.SharedMemory(name=NAME)
layoutb=extract_memory.get_header_format('layout_boxes.h')
shmem_boxes = shared_memory.SharedMemory(name="NW_BUFFER_BOXES")
buf_boxes=shmem_boxes.buf

In [ ]:
nptypes={'double':np.float64, 'uint16_t':np.uint16,'uint8_t':np.uint8}
def shmem_item(shmem1,layout,varname):
    item_info=layout[1][varname]
    data=np.frombuffer(shmem1, dtype=nptypes[item_info['type']], count=item_info['num_elements'], offset=item_info['bytenum_current'])
    if len(data)==1:
        data=data[0]
    return data

class NW_shmem:
    def __init__(self,mem,layout):
        self.mem = mem
        self.layout = layout

    def item(self,name):
        return shmem_item(self.mem, self.layout, name)

shmem_boxes = NW_shmem(buf_boxes,layoutb)
    
num_boxes=shmem_item( buf_boxes, layoutb, 'num_boxes')
centroid_x=shmem_item( buf_boxes, layoutb, 'centroid_x')
centroid_y=shmem_item( buf_boxes, layoutb, 'centroid_y')
reference_x=shmem_item( buf_boxes, layoutb, 'reference_x')
reference_y=shmem_item( buf_boxes, layoutb, 'reference_y')

print( num_boxes, centroid_x[0],  centroid_y[0] )

In [ ]:
shmem_boxes.item("centroid_y")[23]

In [ ]:
spot_displace_x =   reference_x - centroid_x
spot_displace_y = -(reference_y - centroid_y)

In [ ]:
plt.plot( centroid_x[0:num_boxes],centroid_y[0:num_boxes], 'k.' )
plt.gca().invert_yaxis()